<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E3%80%90%E9%87%91%E9%91%B0%E3%80%91%E5%90%88%E4%BD%9C%E7%A4%BE%E7%95%B6%E6%97%A5%E3%80%8CT_point%E3%80%8D%E6%9F%A5%E8%A9%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (使用隱藏)

## 匯入模組

In [ ]:
!pip3 install pygsheets

In [ ]:
!pip install gspread-dataframe==3.2.1

In [ ]:
import pygsheets
gc = pygsheets.authorize(service_file='/content/orderdata-copyto-parttime-5c19ba8fcba4.json') # Gild學長後續請將.json金鑰設定在local路徑
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd

## 讀取資料

### 讀取來自消費者的訂購紀錄

In [ ]:
# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("可寄發取貨通知")
#讀取成 df
Order_data = pd.DataFrame(wks.get_all_records())

### 「商品」對應到「供應商」

In [ ]:
# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1pSHCFR-pHePTHDCMlnkS5vPS6imU_fLBk_r9eYmBuV0/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")
#讀取成 df
Price_data = pd.DataFrame(wks.get_all_records())



In [ ]:
import re
# Assuming pricedata is a DataFrame with your data
prices = Price_data["銷售價格"].apply(lambda x: re.sub(r'\D', '', x)).astype(int)
Price_data["銷售價格"] = prices
Price_data

TypeError: ignored

In [ ]:
Price_data["商品全稱"] = Price_data["供應商"] + Price_data["商品名稱"]
Price_data

## 數據處理

### 加總：各品項當日訂購數量

In [ ]:
# 获取 Price_data 中的商品全称列
product_names = Price_data['商品全稱']

# 遍历 Order_data 中的每一行
for index, row in Order_data.iterrows():
    subtotal = 0  # 初始化小计为0
    for product in product_names:
        quantity = row.get(product, 0)  # 获取数量，如果没有则默认为0
        if quantity == '':
            quantity = 0
        else:
            quantity = float(quantity)  # 强制转换为浮点数
        price = float(Price_data.loc[Price_data['商品全稱'] == product, '銷售價格'].values[0])  # 强制转换为浮点数
        subtotal += quantity * price
    Order_data.at[index, '小計'] = subtotal

# 根据电子邮件地址向每个订购者发送帐单
for index, row in Order_data.iterrows():
    email = row['電子郵件地址']
    name = row['訂購者姓名']
    subtotal = row['小計']
    # 此处添加发送帐单的代码，可以将帐单信息发送到对应的电子邮件地址

# 打印 Order_data，包括小计
Order_data

### 寄出T-point信件

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os


def email_sent(Tosearch):
  # 設定寄件者的電子郵件地址和密碼作為環境變數
  os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
  os.environ["COOP_PASSWORD"] = "coop4s@nthu"
  filtered_data = Order_data[Order_data['訂單編號'] == Tosearch]
  Selected_filtered_data = filtered_data.iloc[:, [2, 3, 5, -2, -1]]
  print("===================================")
  print(Selected_filtered_data)
  print("===================================\n")
  try:
    AreYouSure = int(input("請確定，上述關於該訂單資訊是否正確，如正確請按1 \n"))
    print("===================================\n")
    if AreYouSure == 1 :
        # Your loop for sending emails
        for index, row in filtered_data.iterrows():
            recipient_name = row["訂購者姓名"]
            order_num = row["訂單編號"]
            product_quantities = row[6:-2]
            total_amount = row["小計"]

            # Create email content
            email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，您已付款購買的產品詳情如下，請確認:\n\n"
            email_content += f"===================================\n"
            email_content += f"系統自動生成的訂單編號: {order_num} \n"
            email_content += f"==================================="
            for product, quantity in product_quantities.items():
                if quantity != "" :
                  # Check if the product is in the "商品名稱" column
                  if product in Price_data["商品全稱"].values:
                      # If the product is found, you can retrieve its "銷售價格"
                      sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
                  email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"
            email_content += f"===================================\n"
            email_content += f"付款金額小計: {total_amount} 新台幣\n"
            email_content += f"獲得T-point: {total_amount} 點數\n"


            # Email settings
            sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
            sender_password = "jwsscmoiffdyukmm"
            recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

            subject = "【十里方圓合作社】完成結帳通知"
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = recipient_email
            message["Subject"] = subject
            message.attach(MIMEText(email_content, "plain"))

            try:
                # Connect to SMTP server
                server = smtplib.SMTP("smtp.gmail.com", 587)
                server.starttls()
                server.login(sender_email, sender_password)

                # Send the email
                server.sendmail(sender_email, recipient_email, message.as_string())

                # Close the SMTP server
                server.quit()

                print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
            except Exception as e:
                print(f"Failed to send email to {recipient_name}. Error: {str(e)}")
    else:
        print("停止輸入")
  except ValueError:
    print("停止輸入")

# 工讀生輸入

In [ ]:
# 使用者輸入
Tosearch = int(input("請輸入結帳完成的「訂單編號」："))
email_sent(Tosearch)
